In [1]:
import re
import json
import random
import nltk

nltk.download("punkt")
nltk.download("punkt_tab") 
nltk.download("averaged_perceptron_tagger")
nltk.download("averaged_perceptron_tagger_eng")  
nltk.download("stopwords")

from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words("english"))


[nltk_data] Downloading package punkt to /Users/sheya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/sheya/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sheya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/sheya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/sheya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#data of chatbot

reci_data = {
  "intents": {
    "greeting": {
      "patterns": [r"\bhi\b", r"\bhello\b", r"\bhey\b"],
      "responses": [
        "Hi! I’m Reci. Tell me what ingredients you have, your time limit, or what you feel like eating!"
      ]
    },
    "suggest": {
      "patterns": [
        r"\bwhat should i cook\b", r"\bwhat to cook\b", r"\bdinner\b", r"\bmeal\b",
        r"\bsuggest\b", r"\brecipe\b", r"\bideas\b", r"\bhelp me cook\b"
      ],
      "responses": [
        "Tell me what ingredients,cravings (spicy/healthy/comfort/quick) or amount of time (e.g. 15 mins) you have."
      ]
    },
    "bye": {
      "patterns": [r"\bbye\b", r"\bquit\b", r"\bexit\b", r"\bgoodbye\b"],
      "responses": ["Bye! enjoy your meal 😄"]
    }
  },

  "ingredient_aliases": {
    # noodles 
    "instant noodles": "noodles",
    "maggi": "noodles",
    "ramen": "noodles",
    "udon": "noodles",
    "soba": "noodles",

    # rice noodles 
    "bee hoon": "rice noodles",
    "vermicelli": "rice noodles",
    "rice noodle": "rice noodles",
    "rice noodles": "rice noodles",
    "kway teow": "rice noodles",

    # staples and proteins
    "rice": "rice",
    "egg": "egg",
    "eggs": "egg",
    "chicken": "chicken",
    "beef": "beef",
    "tofu": "tofu",
    "tuna": "tuna",

    # fish and seafood
    "fish": "fish",
    "salmon": "fish",
    "dory": "fish",
    "cod": "fish",
    "tilapia": "fish",
    "prawn": "prawns",
    "prawns": "prawns",
    "shrimp": "prawns",
    "shrimps": "prawns",

    # sauces and others
    "soy sauce": "soy sauce",
    "miso": "miso",
    "mayo": "mayo",
    "cheese": "cheese",
    "tomato": "tomato",
    "garlic": "garlic",
    "onion": "onion",
    "spring onion": "spring onion",
    "scallion": "spring onion",
    "spinach": "spinach",
    "kimchi": "kimchi",
    "butter": "butter",
    "ginger": "ginger",
    "chilli": "chilli",
    "chili": "chilli",
    "lemon": "lemon"
  },

  "recipes": [
    # noodles
    {
      "name": "Garlic Egg Noodles (10-min)",
      "time_mins": 10,
      "tags": ["quick", "comfort"],
      "needs": ["noodles", "egg", "garlic", "soy sauce"],
      "steps": [
        "Boil noodles (2–3 mins). Reserve 2 tbsp noodle water, then drain.",
        "Pan medium heat: 1 tbsp oil. Fry 2 minced garlic cloves (20–30 sec).",
        "Add noodles + 1 tbsp soy sauce + reserved noodle water. Toss 30 sec.",
        "Push noodles aside, scramble 1 egg in the pan, then mix through.",
        "Taste: add pepper/chilli (optional). Serve hot."
      ]
    },
    {
      "name": "Spicy Tuna Mayo Noodles (8-min)",
      "time_mins": 8,
      "tags": ["quick", "spicy"],
      "needs": ["noodles", "tuna", "mayo"],
      "steps": [
        "Boil noodles (2–3 mins). Drain well.",
        "Mix: 2 tbsp mayo + 1/2 can tuna + chilli flakes/sauce (to taste).",
        "Toss noodles with tuna mayo mix until coated.",
        "Optional: add soy sauce (1 tsp) if you want it saltier.",
        "Eat immediately (best creamy)."
      ]
    },
    {
      "name": "Soy Garlic Chicken Noodles (15-min)",
      "time_mins": 15,
      "tags": ["quick"],
      "needs": ["noodles", "chicken", "soy sauce", "garlic"],
      "steps": [
        "Boil noodles (2–3 mins). Drain and set aside.",
        "Slice chicken thin. Pan high: 1 tbsp oil, cook chicken 4–5 mins until done.",
        "Add garlic 20 sec. Add 1.5 tbsp soy sauce + 2 tbsp water. Toss 30 sec.",
        "Add noodles back in. Stir-fry 1–2 mins until coated.",
        "Optional: add chilli. Serve."
      ]
    },
    {
      "name": "Chicken Miso Ramen (12-min)",
      "time_mins": 12,
      "tags": ["quick", "comfort"],
      "needs": ["noodles", "chicken", "miso", "egg"],
      "steps": [
        "Boil noodles (2–3 mins). Drain and set aside.",
        "In a bowl: mix 1 tbsp miso + 3 tbsp hot water until smooth.",
        "Pan medium: cook sliced chicken 4–5 mins with 1 tbsp oil.",
        "Pour in 1 cup water + miso mix. Simmer 2 mins.",
        "Add noodles. Optional: crack in egg and poach 2–3 mins. Serve."
      ]
    },
    {
      "name": "Garlic Butter Prawns Noodles (12-min)",
      "time_mins": 12,
      "tags": ["quick", "comfort"],
      "needs": ["noodles", "prawns", "garlic", "butter"],
      "steps": [
        "Boil noodles (2–3 mins). Reserve 2 tbsp noodle water, drain.",
        "Pan medium-high: melt 1 tbsp butter. Add garlic 20 sec.",
        "Add prawns, cook 2–3 mins until pink.",
        "Add noodles + reserved noodle water. Toss 1 min to coat.",
        "Optional: squeeze lemon. Serve hot."
      ]
    },

    # rice
    {
      "name": "Egg Fried Rice (12-min)",
      "time_mins": 12,
      "tags": ["quick", "comfort"],
      "needs": ["rice", "egg", "soy sauce", "garlic"],
      "steps": [
        "Use cold rice (best). If hot rice, spread on plate 3 mins to dry.",
        "Pan high heat: 1 tbsp oil. Scramble 2 eggs, remove.",
        "Add garlic (1–2 cloves) 20 sec, then add rice. Fry 2–3 mins.",
        "Add 1 tbsp soy sauce + pepper. Return egg, mix 30 sec.",
        "Optional: frozen peas/corn 2 mins. Serve."
      ]
    },
    {
      "name": "Kimchi Egg Rice (10-min)",
      "time_mins": 10,
      "tags": ["spicy", "quick"],
      "needs": ["rice", "egg", "kimchi"],
      "steps": [
        "Pan medium: 1 tbsp oil. Fry kimchi 2 mins.",
        "Add rice, stir-fry 2–3 mins.",
        "Push aside, scramble 1 egg, mix in.",
        "Optional: soy sauce 1 tsp.",
        "Serve hot."
      ]
    },
    {
      "name": "Chicken Rice Bowl (15-min)",
      "time_mins": 15,
      "tags": ["quick"],
      "needs": ["rice", "chicken", "soy sauce", "garlic"],
      "steps": [
        "Slice chicken thin. Pan medium-high: 1 tbsp oil. Cook chicken 5–6 mins.",
        "Add garlic 20 sec, then 1.5 tbsp soy sauce + 2 tbsp water. Simmer 1 min.",
        "Warm rice (microwave 1–2 mins).",
        "Top rice with soy-garlic chicken and sauce.",
        "Optional: add spring onion/chilli. Serve."
      ]
    },
    {
      "name": "Tuna Mayo Rice Bowl (5-min, No-Cook)",
      "time_mins": 5,
      "tags": ["quick"],
      "needs": ["rice", "tuna", "mayo"],
      "steps": [
        "Drain canned tuna.",
        "Mix: tuna + 1–2 tbsp mayo + pepper (optional).",
        "Warm rice if needed.",
        "Top rice with tuna mix.",
        "Optional: add soy sauce drizzle + spring onion."
      ]
    },
    {
      "name": "Prawn Fried Rice (15-min)",
      "time_mins": 15,
      "tags": ["quick"],
      "needs": ["rice", "prawns", "egg", "garlic", "soy sauce"],
      "steps": [
        "Pan high: 1 tbsp oil. Cook prawns 2–3 mins, remove.",
        "Scramble 1–2 eggs, remove.",
        "Add garlic 20 sec, then add rice. Stir-fry 2–3 mins.",
        "Add 1 tbsp soy sauce, return prawns + egg. Toss 1 min.",
        "Optional: spring onion on top. Serve."
      ]
    },

    # rice noodles
    {
      "name": "Quick Bee Hoon Stir-Fry (15-min)",
      "time_mins": 15,
      "tags": ["quick", "comfort"],
      "needs": ["rice noodles", "egg", "garlic", "soy sauce"],
      "steps": [
        "Soak rice noodles in hot water 3–5 mins until soft, then drain well.",
        "Pan medium-high: 1 tbsp oil. Fry garlic 20 sec.",
        "Add noodles + 1 tbsp soy sauce + 2 tbsp water. Toss 1–2 mins.",
        "Push aside, scramble 1 egg, mix through.",
        "Optional: add chilli/spring onion. Serve."
      ]
    },
    {
      "name": "Garlic Prawns Bee Hoon (12-min)",
      "time_mins": 12,
      "tags": ["quick"],
      "needs": ["rice noodles", "prawns", "garlic", "soy sauce"],
      "steps": [
        "Soak rice noodles in hot water 3–5 mins. Drain well.",
        "Pan medium-high: 1 tbsp oil. Add garlic 20 sec.",
        "Add prawns, cook 2–3 mins until pink.",
        "Add noodles + 1 tbsp soy sauce + 2 tbsp water. Toss 1–2 mins.",
        "Optional: spring onion. Serve."
      ]
    },
    {
      "name": "Ginger Fish Bee Hoon Soup (15-min)",
      "time_mins": 15,
      "tags": ["healthy", "comfort"],
      "needs": ["rice noodles", "fish", "ginger"],
      "steps": [
        "Soak rice noodles 3–5 mins, drain.",
        "Boil 2 cups water. Add sliced ginger. Simmer 3 mins.",
        "Add fish pieces. Simmer 4–5 mins until cooked.",
        "Add rice noodles and heat 1 min.",
        "Optional: add soy sauce to taste. Serve hot."
      ]
    },

    # proteins
    {
      "name": "Honey Soy Chicken (18-min, One-Pan)",
      "time_mins": 18,
      "tags": ["quick", "comfort"],
      "needs": ["chicken", "soy sauce", "garlic"],
      "steps": [
        "Mix sauce: 1.5 tbsp soy sauce + 1 tbsp honey/sugar + 3 tbsp water.",
        "Pan medium-high: 1 tbsp oil. Cook chicken 3 mins per side.",
        "Add garlic 20 sec, pour sauce in.",
        "Reduce heat, simmer 6–8 mins until glossy and cooked through.",
        "Serve with rice/noodles."
      ]
    },
    {
      "name": "Spicy Garlic Chicken Stir-Fry (15-min)",
      "time_mins": 15,
      "tags": ["spicy", "quick"],
      "needs": ["chicken", "garlic", "soy sauce", "chilli"],
      "steps": [
        "Slice chicken thin. Pan high: 1 tbsp oil. Cook chicken 5–6 mins.",
        "Add garlic 20 sec + chilli (to taste).",
        "Add 1 tbsp soy sauce + 2 tbsp water. Toss 1 min.",
        "Optional: add onion for 1–2 mins.",
        "Serve with rice/noodles."
      ]
    },
    {
      "name": "Soy Ginger Pan-Fried Fish (12-min)",
      "time_mins": 12,
      "tags": ["quick", "healthy"],
      "needs": ["fish", "soy sauce", "ginger"],
      "steps": [
        "Pat fish dry. Pan medium-high: 1 tbsp oil.",
        "Cook fish 3–4 mins per side (until flaky).",
        "Add sliced ginger 20–30 sec.",
        "Add 1 tbsp soy sauce + 2 tbsp water. Simmer 1 min.",
        "Serve with rice/noodles."
      ]
    },
    {
      "name": "Garlic Butter Fish (12-min)",
      "time_mins": 12,
      "tags": ["quick", "comfort"],
      "needs": ["fish", "garlic", "butter", "lemon"],
      "steps": [
        "Pat fish dry. Pan medium-high: 1 tbsp oil. Cook fish 3–4 mins per side.",
        "Lower heat. Add 1 tbsp butter + garlic 20 sec.",
        "Spoon butter over fish for 30–60 sec.",
        "Optional: squeeze lemon.",
        "Serve with rice/noodles."
      ]
    },
    {
      "name": "Honey Garlic Prawns (10-min)",
      "time_mins": 10,
      "tags": ["quick"],
      "needs": ["prawns", "garlic", "soy sauce"],
      "steps": [
        "Mix sauce: 1 tbsp soy sauce + 1 tbsp honey/sugar + 2 tbsp water.",
        "Pan medium-high: 1 tbsp oil. Add garlic 20 sec.",
        "Add prawns, cook 2–3 mins until pink.",
        "Pour sauce, toss 1–2 mins until glossy.",
        "Serve with rice/noodles."
      ]
    },
    {
      "name": "Prawn Omelette Rice (12-min)",
      "time_mins": 12,
      "tags": ["quick", "comfort"],
      "needs": ["prawns", "egg", "onion", "rice"],
      "steps": [
        "Chop onion. Pan medium: 1 tbsp oil. Cook prawns 2 mins, add onion 1 min.",
        "Beat 2–3 eggs, pour in. Cook 2–3 mins until set.",
        "Flip or fold, cook 1 min more.",
        "Warm rice.",
        "Serve omelette over rice."
      ]
    },

    
    {
      "name": "Honey Soy Beef Stir-Fry (15-min)",
      "time_mins": 15,
      "tags": ["quick"],
      "needs": ["beef", "soy sauce", "garlic"],
      "steps": [
        "Slice beef thin. Mix sauce: 1 tbsp soy sauce + 1 tbsp honey/sugar + 2 tbsp water.",
        "Pan high: 1 tbsp oil. Sear beef 2–3 mins until browned.",
        "Add garlic 20 sec, pour sauce. Simmer 2–3 mins until glossy.",
        "Optional: add onion/spinach last 1–2 mins.",
        "Serve with rice or noodles."
      ]
    },
    {
      "name": "Tomato Cheese Pasta (18-min)",
      "time_mins": 18,
      "tags": ["comfort"],
      "needs": ["pasta", "tomato", "cheese", "garlic"],
      "steps": [
        "Boil pasta (follow pack time). Reserve 1/4 cup pasta water.",
        "Pan medium: 1 tbsp oil. Fry garlic 20 sec.",
        "Add chopped tomato (or tomato sauce). Cook 4–5 mins until soft.",
        "Add pasta + reserved water. Toss until saucy.",
        "Top cheese, melt 1 min. Serve."
      ]
    },
    {
      "name": "Tofu Soy Bowl (12-min)",
      "time_mins": 12,
      "tags": ["healthy"],
      "needs": ["tofu", "soy sauce", "garlic"],
      "steps": [
        "Pat tofu dry, cube it.",
        "Pan medium-high: 1 tbsp oil. Brown tofu 6–8 mins, turning.",
        "Add garlic 20 sec, then 1 tbsp soy sauce + 2 tbsp water.",
        "Simmer 1–2 mins until coated.",
        "Serve with rice/noodles + optional cucumber."
      ]
    }
  ]
}

with open("reci_data.json", "w", encoding="utf-8") as f:
    json.dump(reci_data, f, indent=2)

print(" Saved reci_data.json")

 Saved reci_data.json


In [3]:
RECIPE_BANK = [
    # chicken
    {
        "name": "Chicken Garlic Soy Stir-Fry (Rice)",
        "tags": {"quick", "comfort"},
        "needs": {"chicken"},
        "diet_ok": {"halal", "none", "gluten-free"},
        "time": 15,
        "steps": [
            "Slice chicken thinly. Season with salt and pepper.",
            "Heat pan medium-high with 1 tbsp oil. Stir-fry chicken 4–6 mins.",
            "Add 2 minced garlic cloves. Stir 30 sec.",
            "Add 1.5 tbsp soy sauce + 2 tbsp water + 1 tsp sugar.",
            "Serve with rice."
        ]
    },
    {
        "name": "Honey Soy Chicken (One-Pan)",
        "tags": {"quick", "comfort"},
        "needs": {"chicken"},
        "diet_ok": {"halal", "none"},
        "time": 18,
        "steps": [
            "Mix soy sauce, honey, lemon/vinegar, water.",
            "Pan medium-high, oil 1 tbsp. Brown chicken.",
            "Add sauce, simmer 6–8 mins until glazed.",
            "Flip once.",
            "Serve with rice/noodles."
        ]
    },
    {
        "name": "Spicy Chili Chicken",
        "tags": {"quick", "spicy"},
        "needs": {"chicken"},
        "diet_ok": {"halal", "none"},
        "time": 15,
        "steps": [
            "Slice chicken thin.",
            "Pan high heat, oil 1 tbsp. Cook chicken 5 mins.",
            "Add garlic and chili flakes.",
            "Add soy sauce and a splash of water.",
            "Serve hot."
        ]
    },
    {
        "name": "Chicken Fried Rice",
        "tags": {"quick", "comfort"},
        "needs": {"chicken", "rice", "egg"},
        "diet_ok": {"halal", "none"},
        "time": 15,
        "steps": [
            "Cook diced chicken. Remove.",
            "Scramble egg. Remove.",
            "Fry rice 2–3 mins.",
            "Return chicken + egg, add soy sauce.",
            "Serve hot."
        ]
    },

    # fish
    {
        "name": "Soy Ginger Pan-Fried Fish",
        "tags": {"quick", "healthy"},
        "needs": {"fish"},
        "diet_ok": {"halal", "none"},
        "time": 12,
        "steps": [
            "Season fish lightly.",
            "Pan medium high, oil 1 tbsp. Cook 3–4 mins per side.",
            "Add ginger.",
            "Add soy sauce + 2 tbsp water.",
            "Serve with rice."
        ]
    },
    {
        "name": "Garlic Butter Fish",
        "tags": {"quick", "comfort"},
        "needs": {"fish"},
        "diet_ok": {"halal", "none"},
        "time": 12,
        "steps": [
            "Pan medium-high, oil 1 tbsp. Cook fish.",
            "Lower heat, add butter + garlic.",
            "Baste fish 1 min.",
            "Optional lemon.",
            "Serve hot."
        ]
    },
    {
        "name": "Crispy Pan-Fried Fish",
        "tags": {"quick"},
        "needs": {"fish"},
        "diet_ok": {"halal", "none"},
        "time": 15,
        "steps": [
            "Pat fish dry, season.",
            "Pan high heat, oil 1 tbsp.",
            "Cook fish until golden.",
            "Flip once only.",
            "Serve with rice."
        ]
    },

    # prawn
    {
        "name": "Honey Garlic Prawns",
        "tags": {"quick"},
        "needs": {"prawns"},
        "diet_ok": {"halal", "none"},
        "time": 10,
        "steps": [
            "Mix soy sauce, honey, water.",
            "Pan medium high, oil 1 tbsp. Add garlic.",
            "Add prawns, cook 2–3 mins.",
            "Pour sauce, toss.",
            "Serve hot."
        ]
    },
    {
        "name": "Chili Prawn Stir-Fry",
        "tags": {"quick", "spicy"},
        "needs": {"prawns"},
        "diet_ok": {"halal", "none"},
        "time": 10,
        "steps": [
            "Pan high heat, oil 1 tbsp.",
            "Add prawns, cook 2 mins.",
            "Add garlic and chili.",
            "Add soy sauce.",
            "Serve immediately."
        ]
    },
    {
        "name": "Prawn Fried Rice",
        "tags": {"quick", "comfort"},
        "needs": {"prawns", "rice", "egg"},
        "diet_ok": {"halal", "none"},
        "time": 15,
        "steps": [
            "Cook prawns, remove.",
            "Scramble egg, remove.",
            "Fry rice 2–3 mins.",
            "Return prawns and egg.",
            "Serve hot."
        ]
    },

    # noodles
    {
        "name": "Chicken Soy Noodles",
        "tags": {"quick"},
        "needs": {"chicken", "noodles"},
        "diet_ok": {"halal", "none"},
        "time": 12,
        "steps": [
            "Boil noodles, drain.",
            "Cook chicken in pan.",
            "Add noodles and soy sauce.",
            "Toss for 1 min.",
            "Serve hot."
        ]
    },
    {
        "name": "Prawn Garlic Noodles",
        "tags": {"quick"},
        "needs": {"prawns", "noodles"},
        "diet_ok": {"halal", "none"},
        "time": 12,
        "steps": [
            "Boil noodles, drain.",
            "Pan oil and garlic.",
            "Add prawns, cook 2 mins.",
            "Add noodles and soy sauce.",
            "Toss and serve."
        ]
    },

    # rice
    {
        "name": "Egg Fried Rice (Quick)",
        "tags": {"quick", "cheap", "comfort"},
        "needs": {"egg", "rice"},
        "diet_ok": {"halal", "none"},
        "time": 12,
        "steps": [
            "Scramble eggs, remove.",
            "Fry rice 2–3 mins.",
            "Add soy sauce and pepper.",
            "Return eggs.",
            "Serve."
        ]
    },
    {
        "name": "Tuna Mayo Rice Bowl (No Cook)",
        "tags": {"quick", "cheap"},
        "needs": {"tuna", "rice"},
        "diet_ok": {"halal", "none"},
        "time": 5,
        "steps": [
            "Mix tuna + mayo.",
            "Warm rice.",
            "Top rice with tuna.",
            "Done."
        ]
    },
    {
        "name": "Tofu Veg Bowl",
        "tags": {"healthy", "quick", "vegetarian"},
        "needs": {"tofu"},
        "diet_ok": {"vegetarian", "vegan", "halal", "none"},
        "time": 18,
        "steps": [
            "Crisp tofu.",
            "Add veg and water.",
            "Add soy sauce.",
            "Toss.",
            "Serve with rice or noodles."
        ]
    }
]

print(f" Loaded {len(RECIPE_BANK)} recipes")

 Loaded 15 recipes


In [4]:
STOP_INGREDIENT_WORDS = {"dinner", "meal", "time", "minutes", "minute", "mins", "min", "want", "need"}

def preprocess_text(text):
    # lowercase all text for handling
    text = text.lower()

    # remove non characters
    text = re.sub(r"[^a-z0-9\s]", " ", text)

    # tokenize
    tokens = nltk.word_tokenize(text)
    return tokens

def pos_tag_tokens(tokens):
    return nltk.pos_tag(tokens)

def extract_entities(tokens, tagged):
    info = {
        "ingredients": [],
        "craving": None,
        "minutes": None,
        "diet": None,
        "wants_more": False
    }

    raw = " ".join(tokens)

    # minutes from regex
    m = re.search(r"\b(\d{1,3})\s*(?:min|mins|minute|minutes)\b", raw)
    if m:
        info["minutes"] = int(m.group(1))
    elif "quick" in tokens or "fast" in tokens:
        info["minutes"] = 15

    # cravings
    cravings = {"spicy", "healthy", "comfort", "cheap", "quick", "light", "crispy"}
    for t in tokens:
        if t in cravings:
            info["craving"] = t
            break

    # diet 
    if "vegan" in tokens:
        info["diet"] = "vegan"
    elif "vegetarian" in tokens:
        info["diet"] = "vegetarian"
    elif "halal" in tokens:
        info["diet"] = "halal"
    elif "gluten" in tokens and "free" in tokens:
        info["diet"] = "gluten-free"
    elif "pork" in tokens and "no" in tokens:
        info["diet"] = "no pork"

    # wants more recipe?
    if re.search(r"\byes\b|\banother\b|\bmore\b", raw):
        info["wants_more"] = True

    # ingredient extraction using pos tags
    for word, tag in tagged:
        if tag in ("NN", "NNS"):
            if word not in STOP_INGREDIENT_WORDS:
                info["ingredients"].append(word)

    seen = set()
    cleaned = []
    for ing in info["ingredients"]:
        if ing not in seen:
            seen.add(ing)
            cleaned.append(ing)

    info["ingredients"] = cleaned
    return info

    # remove duplicates 
    seen = set()
    cleaned = []
    for x in info["ingredients"]:
        if x not in seen:
            seen.add(x)
            cleaned.append(x)
    info["ingredients"] = cleaned

    return info

In [5]:
def load_intents(path="intents.json"):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

INTENTS = load_intents()

def detect_intent(user_input):
    for intent_name, intent_data in INTENTS.items():
        for pattern in intent_data["patterns"]:
            if re.search(pattern, user_input, re.IGNORECASE):
                return intent_name
    return None

def recipe_is_allowed(recipe, memory):
    diet = memory.get("diet") or "none"
    if diet == "no pork":
        return True  
    return (diet in recipe["diet_ok"]) or ("none" in recipe["diet_ok"])

def recommend_recipe(memory):
    craving = memory.get("craving")
    ingredients = set(memory.get("ingredients", []))
    already = set(memory.get("suggested", []))
    max_mins = memory.get("minutes")

    candidates = []
    for r in RECIPE_BANK:
        if r["name"] in already:
            continue
        if not recipe_is_allowed(r, memory):
            continue
        if max_mins is not None and r["time"] > max_mins:
            continue

        score = 0
        # craving match
        if craving and craving in r["tags"]:
            score += 2
        # ingredient overlap
        score += len(ingredients.intersection(r["needs"]))

        # only keep recipes that have some chance
        if score > 0 or (max_mins is not None and r["time"] <= max_mins):
            candidates.append((score, r))

    if not candidates:
        return None

    candidates.sort(key=lambda x: x[0], reverse=True)
    # if ties, random among top few for variety
    top_score = candidates[0][0]
    top = [r for s, r in candidates if s == top_score]
    return random.choice(top)

def format_steps(steps):
    # more precise looking steps with numbering
    return "\n".join([f"{i+1}. {s}" for i, s in enumerate(steps)])

def generate_response(intent, memory):
    # suggest intent returns a recipe
    if intent in {"suggest", "craving", "time", "diet", "ingredients"}:
        recipe = recommend_recipe(memory)
        if recipe is None:
            return ("I’m out of matches 😅 Try searching using a time limit "
                    "(e.g., '15 mins'), or a craving (spicy/healthy/comfort).")

        memory["suggested"].append(recipe["name"])  
        why_bits = []
        if memory.get("craving"):
            why_bits.append(f"craving={memory['craving']}")
        if memory.get("ingredients"):
            why_bits.append(f"ingredients={', '.join(memory['ingredients'][:5])}")

        why = " / ".join(why_bits) if why_bits else "your preferences"

        return (
            f"You can make: **{recipe['name']}**\n"
            f"Time: ~{recipe['time']} mins\n"
            f"Why: matches {why}.\n"
            f"Steps:\n{format_steps(recipe['steps'])}\n"
            f"Want another suggestion? (yes/no)"
        )

    #use JSON
    templates = INTENTS[intent]["responses"]
    template = random.choice(templates)

    template = template.replace("{craving}", str(memory.get("craving", "something nice")))
    template = template.replace("{minutes}", str(memory.get("minutes", 20)))
    template = template.replace("{diet}", str(memory.get("diet", "no restrictions")))
    template = template.replace("{ingredients}", ", ".join(memory.get("ingredients", [])) or "none yet")

    return template

from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words("english"))

def build_ingredient_vocab(recipe_bank):
    """Build a set of known ingredients from RECIPE_BANK."""
    vocab = set()
    for r in recipe_bank:
        for ing in r.get("needs", []):
            vocab.add(ing.lower())
    return vocab

INGREDIENT_VOCAB = build_ingredient_vocab(RECIPE_BANK)

def extract_entities(tokens, tagged):
    """
    Extract structured info:
    - ingredients: only from known ingredient vocab (prevents 'i', 'cook', etc.)
    - craving: from a keyword list
    - minutes: from regex
    - diet: from keyword list
    """
    info = {
        "ingredients": [],
        "craving": None,
        "minutes": None,
        "diet": None
    }

    # minutes extraction 
    joined = " ".join(tokens)
    m = re.search(r"\b(\d{1,3})\s*(min|mins|minute|minutes)\b", joined)
    if m:
        info["minutes"] = m.group(1)

    # craving keywords
    craving_keywords = {"spicy", "healthy", "comfort", "cheap", "quick", "light", "crispy"}
    for t in tokens:
        if t in craving_keywords:
            info["craving"] = t

    # diet keywords
    if "gluten" in tokens and "free" in tokens:
        info["diet"] = "gluten-free"
    diet_keywords = {"vegetarian", "vegan", "halal", "no", "pork"} 
    for t in tokens:
        if t in diet_keywords:
            if t == "vegetarian":
                info["diet"] = "vegetarian"
            elif t == "vegan":
                info["diet"] = "vegan"
            elif t == "halal":
                info["diet"] = "halal"
            

    # ingredient extraction 
    clean = []
    for t in tokens:
        if t in STOPWORDS:
            continue
        if not re.fullmatch(r"[a-z]+", t):
            continue
        if len(t) < 2:
            continue
        clean.append(t)

    # accept token as ingredient only if in ingredient vocab
    for t in clean:
        if t in INGREDIENT_VOCAB:
            info["ingredients"].append(t)

    # deduplicate while preserving order
    seen = set()
    deduped = []
    for ing in info["ingredients"]:
        if ing not in seen:
            seen.add(ing)
            deduped.append(ing)
    info["ingredients"] = deduped

    return info

In [6]:
import re
import json
import random
from nltk.corpus import stopwords
import nltk

STOPWORDS = set(stopwords.words("english"))

with open("reci_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

intents = data["intents"]
aliases = data["ingredient_aliases"]
recipes = data["recipes"]

vocab = set()
for recipe in recipes:
    for ing in recipe.get("needs", []):
        vocab.add(ing.lower())
for k, v in aliases.items():
    vocab.add(k.lower())
    vocab.add(v.lower())

# trying to match user input to known patterns
def find_intent(text):
    for name, intent in intents.items():
        for pattern in intent["patterns"]:
            if re.search(pattern, text, re.IGNORECASE):
                return name
    return None

def get_minutes(text):
    match = re.search(r"(\d+)\s*(min|minute|minutes)", text.lower())
    if match:
        return int(match.group(1))
    return None

def get_craving(text):
    tags = ["spicy", "healthy", "comfort", "quick"]
    text = text.lower()
    for tag in tags:
        if tag in text:
            return tag
    return None

def process_text(text):
    text = re.sub(r"[^a-z0-9\s]", " ", text.lower())
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

def get_ingredients(text):
    tokens = process_text(text)
    found = set()
    junk_words = {
        "i","im","i'm","me","my","mine","you","your","yours",
        "want","wanna","like","feel","having","cook","make","eat",
        "dinner","meal","food","tonight","today","please",
        "mins","min","minutes","minute"
    }
    for n in (3, 2):
        for i in range(len(tokens) - n + 1):
            phrase = " ".join(tokens[i:i+n])
            if phrase in aliases or phrase in vocab:
                found.add(aliases.get(phrase, phrase))
    for word in tokens:
        if word in STOPWORDS or word in junk_words:
            continue
        if word in vocab or word in aliases:
            found.add(aliases.get(word, word))
    return sorted(found)

def suggest_recipe(mem):
    ing_set = set(mem["ingredients"])
    craving = mem["craving"]
    mins = mem["minutes"]
    prev = set(mem["suggested"])

    results = []
    for r in recipes:
        if r["name"] in prev:
            continue
        need = set([x.lower() for x in r.get("needs", [])])
        if ing_set and not ing_set & need:
            continue
        if mins is not None and r.get("time_mins", 999) > mins:
            continue
        score = 0
        score += len(ing_set & need) * 10
        if craving and craving in r.get("tags", []):
            score += 5
        results.append((score, r))

    if not results:
        return None
    results.sort(key=lambda x: x[0], reverse=True)
    return results[0][1]

def print_recipe(recipe, mem):
    ing = ", ".join(mem["ingredients"]) if mem["ingredients"] else "none"
    craving = mem["craving"] or "none"
    mins = mem["minutes"]
    time_line = f"~{recipe['time_mins']} mins"
    if mins:
        time_line += f" (fits your {mins} mins)"
    print(f"You can make: {recipe['name']}")
    print(f"Time: {time_line}")
    print(f"Why: ingredients = {ing}, craving = {craving}")
    print("Steps:")
    for idx, step in enumerate(recipe["steps"]):
        print(f"{idx+1}. {step}")
    print("Want another suggestion? (yes/no)")

def reset_memory():
    return {"ingredients": [], "craving": None, "minutes": None, "suggested": []}

# main loop
def chatbot():
    mem = reset_memory()
    waiting = False

    print("Reci: Hi! I can help with meal ideas. Say 'exit' to quit.")
    print("Reci: Give me ingredients, cravings (spicy/healthy/etc), or time like '15 minutes'")

    while True:
        user = input("You: ").strip()
        if not user:
            continue

        if re.search(r"\b(exit|quit|bye|goodbye)\b", user, re.IGNORECASE):
            print("Reci:", random.choice(intents["bye"]["responses"]))
            break

        if waiting:
            if re.fullmatch(r"\s*yes\s*", user, re.IGNORECASE):
                recipe = suggest_recipe(mem)
                if recipe:
                    mem["suggested"].append(recipe["name"])
                    print_recipe(recipe, mem)
                else:
                    print("Reci: I'm out of ideas Try different inputs.")
                    waiting = False
                continue
            elif re.fullmatch(r"\s*no\s*", user, re.IGNORECASE):
                mem = reset_memory()
                print("Reci: Okay! What's next? New ingredients or craving?")
                waiting = False
                continue
            waiting = False

        # process inputs
        old = (tuple(mem["ingredients"]), mem["craving"], mem["minutes"])

        mins = get_minutes(user)
        if mins:
            mem["minutes"] = mins

        crv = get_craving(user)
        if crv:
            mem["craving"] = crv

        ings = get_ingredients(user)
        for i in ings:
            if i not in mem["ingredients"]:
                mem["ingredients"].append(i)

        new = (tuple(mem["ingredients"]), mem["craving"], mem["minutes"])
        if new != old:
            mem["suggested"] = []

        # check intents
        intent = find_intent(user)
        if intent == "greeting":
            print("Reci:", random.choice(intents["greeting"]["responses"]))
            continue
        elif intent == "suggest":
            print("Reci:", random.choice(intents["suggest"]["responses"]))
            continue

        recipe = suggest_recipe(mem)
        if recipe:
            mem["suggested"].append(recipe["name"])
            print_recipe(recipe, mem)
            waiting = True
        else:
            print("Reci: Couldn't find a recipei crave Try more/different ingredients.")

# run it
chatbot()


Reci: Hi! I can help with meal ideas. Say 'exit' to quit.
Reci: Give me ingredients, cravings (spicy/healthy/etc), or time like '15 minutes'


You:  i want something spicy with chicken


You can make: Spicy Garlic Chicken Stir-Fry (15-min)
Time: ~15 mins
Why: ingredients = chicken, craving = spicy
Steps:
1. Slice chicken thin. Pan high: 1 tbsp oil. Cook chicken 5–6 mins.
2. Add garlic 20 sec + chilli (to taste).
3. Add 1 tbsp soy sauce + 2 tbsp water. Toss 1 min.
4. Optional: add onion for 1–2 mins.
5. Serve with rice/noodles.
Want another suggestion? (yes/no)


You:  yes


You can make: Soy Garlic Chicken Noodles (15-min)
Time: ~15 mins
Why: ingredients = chicken, craving = spicy
Steps:
1. Boil noodles (2–3 mins). Drain and set aside.
2. Slice chicken thin. Pan high: 1 tbsp oil, cook chicken 4–5 mins until done.
3. Add garlic 20 sec. Add 1.5 tbsp soy sauce + 2 tbsp water. Toss 30 sec.
4. Add noodles back in. Stir-fry 1–2 mins until coated.
5. Optional: add chilli. Serve.
Want another suggestion? (yes/no)


...

You can make: Tuna Mayo Rice Bowl (5-min, No-Cook)
Time: ~5 mins (fits your 10 mins)
Why: ingredients = fish, prawns, rice, craving = spicy
Steps:
1. Drain canned tuna.
2. Mix: tuna + 1–2 tbsp mayo + pepper (optional).
3. Warm rice if needed.
4. Top rice with tuna mix.
5. Optional: add soy sauce drizzle + spring onion.
Want another suggestion? (yes/no)


You:  im feeling something spicy


You can make: Honey Garlic Prawns (10-min)
Time: ~10 mins (fits your 10 mins)
Why: ingredients = fish, prawns, rice, craving = spicy
Steps:
1. Mix sauce: 1 tbsp soy sauce + 1 tbsp honey/sugar + 2 tbsp water.
2. Pan medium-high: 1 tbsp oil. Add garlic 20 sec.
3. Add prawns, cook 2–3 mins until pink.
4. Pour sauce, toss 1–2 mins until glossy.
5. Serve with rice/noodles.
Want another suggestion? (yes/no)


You:  exit


Reci: Bye! enjoy your meal 😄
